In [1]:
import numpy as np
import pandas as pd
from scipy.stats import truncnorm, poisson
from scipy.optimize import minimize
import collections

# Usando um parâmetro por jogador

In [2]:
def generate_games():
    games = [[chr(i + 65) for i in range(20)] for j in range(20)]
    for i in range(20):
        games[i].pop(i)
        
    return games

def generate_players(temps = 15):
    players = list(np.random.normal(70, 10, 220))
    players = [round(player, 1) if player < 100 else 100 for player in players]
    clubs = {}
    for year in range(temps):
        line_up = {}
        if type(players) != list:
            np.random.shuffle(list(players.flat))

        players = np.reshape(players, (20, 11))
        for club in range(20):
            line_up[chr(65 + club)] = players[club]

        clubs[year] = line_up
        
    return clubs

def find_median(array):
    arraycopy = array.copy()
    arraycopy.sort()
    if len(arraycopy) % 2 == 0:
        return (arraycopy[len(arraycopy) // 2 - 1] + arraycopy[len(arraycopy) // 2])/2
    else:
        return arraycopy[len(arraycopy) // 2]
    
def find_forces(clubs, year = 0):
    atk_forces = []
    def_forces = []
    for club in clubs[year]:
        atk_forces.append(np.sum(clubs[year][club]))
        def_forces.append(np.log(np.prod(clubs[year][club])))
            
    atk_med = find_median(atk_forces)
    def_med = find_median(def_forces)
    
    for i in range(len(atk_forces)):
        atk_forces[i] = atk_forces[i] / atk_med * 2
        def_forces[i] /= def_med
            
    return atk_forces, def_forces

def model(club1, club2, sims = 10000):
    '''
    receives 2 clubs (forces) and returns the most likely result, probability of each result and probability for the game
    '''
    results = np.zeros((6, 6), dtype = int)
    probs = [0, 0, 0]
    for i in range(sims):
        goals1 = poisson.rvs(club1[0] - club2[1])
        goals2 = poisson.rvs(club2[0] - club1[1])
        if goals1 > 5:
            goals1 = 5
        if goals2 > 5:
            goals2 = 5
        
        results[goals1, goals2] += 1
        
        if goals1 > goals2:
            probs[0] += 1
        elif goals1 == goals2:
            probs[1] += 1
        else:
            probs[2] += 1
    
    for i in range(3):
        probs[i] /= sims
        
    result = [0, 0] # Home - away
    for i in range(len(results)):
        for j in range(len(results[i])):
            if results[i, j] > results[result[0], result[1]]:
                result = [i, j]
            elif results[i, j] == results[result[0], result[1]]:
                # rare cases
                if np.random.random() > 0.5:
                    result = [i, j]
    
    return results, result, probs

In [3]:
def championship(clubs, sims = 10000, year = 0):
    atk_forces, def_forces = find_forces(clubs, year = year)
    games = generate_games()
    results = []
    table = np.zeros((20, 8), dtype = int)
    # lines = clubs
    # columns = {points, games, wins, draws, defeats, goals for, goals against, goal difference}
    
    for i in range(len(games)):
        home = i
        for away in games[i]:
            away = ord(away) - 65
            club1 = (atk_forces[home], def_forces[home])
            club2 = (atk_forces[away], def_forces[away])
            _, result, _ = model(club1, club2, sims = sims)
            results.append([chr(home + 65), result[0], result[1], chr(away + 65)])
            
            # games
            table[home, 1] += 1
            table[away, 1] += 1
            
            # goals
            table[home, 5] += result[0]
            table[away, 5] += result[1]
            table[home, 6] += result[1]
            table[away, 6] += result[0]
            table[home, 7] += result[0] - result[1]
            table[away, 7] += result[1] - result[0]
            
            # results
            if result[0] > result[1]:
                table[home, 2] += 1
                table[home, 0] += 3
                table[away, 4] += 1
            elif result[0] == result[1]:
                table[home, 3] += 1
                table[away, 3] += 1
                table[home, 0] += 1
                table[away, 0] += 1
            else:
                table[home, 4] += 1
                table[away, 2] += 1
                table[away, 0] += 3
    
    columns_names = ['points', 'games', 'wins', 'draws', 'defeats', 'goals for', 'goals against', 'goal difference']
    
    table = pd.DataFrame(table,
                        index = [chr(i + 65) for i in range(20)],
                        columns = columns_names)
    table.sort_values(['points',  'wins', 'goal difference', 'goals for'],
                      axis = 0,
                      ascending = False,
                      inplace = True)
    
    return table, results

In [4]:
clubs = generate_players(temps = 1)
atk_forces, def_forces = find_forces(clubs)

# exemplo
club1 = (atk_forces[ord('A') - 65], def_forces[ord('A') - 65])
club2 = (atk_forces[ord('B') - 65], def_forces[ord('B') - 65])
results, result, probs = model(club1, club2, sims = 10000)

print('Forças do clube 1:', club1)
print('Forças do clube 2:', club2)
print()
print('Quantidade de vezes que o placar foi i x j (linha 5 e coluna 5 representam 5 ou mais gols):')
print(results)
print()
print('O resultado mais provável é {} x {}.'.format(result[0], result[1]))
print('Além disso, a probabilidade do mandante é {}, enquanto a do visitante é de {} e a de empate é {}.'.format(probs[0], probs[2], probs[1]))

Forças do clube 1: (2.079666753449622, 1.0105644169276764)
Forças do clube 2: (2.010934652434262, 1.0011488205496692)

Quantidade de vezes que o placar foi i x j (linha 5 e coluna 5 representam 5 ou mais gols):
[[1289 1252  655  224   48    9]
 [1349 1308  624  217   55   12]
 [ 778  730  346  120   31   10]
 [ 258  265  116   45   12    0]
 [  62   74   41   11    4    0]
 [  23   15   14    3    0    0]]

O resultado mais provável é 1 x 0.
Além disso, a probabilidade do mandante é 0.3739, enquanto a do visitante é de 0.3269 e a de empate é 0.2992.


In [5]:
df, results = championship(clubs, sims = 1000)
df

,points,games,wins,draws,defeats,goals for,goals against,goal difference
G,74,38,19,17,2,32,15,17
A,72,38,17,21,0,35,18,17
I,69,38,17,18,3,30,16,14
Q,67,38,15,22,1,36,22,14
J,59,38,13,20,5,32,24,8
F,50,38,11,17,10,23,22,1
H,49,38,10,19,9,24,23,1
S,48,38,11,15,12,20,21,-1
E,48,38,9,21,8,24,23,1
O,46,38,10,16,12,17,19,-2


In [6]:
results

[['A', 1, 1, 'B'],
 ['A', 1, 1, 'C'],
 ['A', 1, 0, 'D'],
 ['A', 1, 0, 'E'],
 ['A', 1, 1, 'F'],
 ['A', 1, 1, 'G'],
 ['A', 1, 0, 'H'],
 ['A', 1, 1, 'I'],
 ['A', 1, 0, 'J'],
 ['A', 1, 0, 'K'],
 ['A', 1, 0, 'L'],
 ['A', 1, 0, 'M'],
 ['A', 0, 0, 'N'],
 ['A', 1, 0, 'O'],
 ['A', 1, 0, 'P'],
 ['A', 1, 1, 'Q'],
 ['A', 1, 1, 'R'],
 ['A', 1, 0, 'S'],
 ['A', 1, 0, 'T'],
 ['B', 1, 1, 'A'],
 ['B', 1, 1, 'C'],
 ['B', 0, 0, 'D'],
 ['B', 1, 1, 'E'],
 ['B', 0, 1, 'F'],
 ['B', 0, 0, 'G'],
 ['B', 1, 1, 'H'],
 ['B', 0, 1, 'I'],
 ['B', 1, 1, 'J'],
 ['B', 0, 0, 'K'],
 ['B', 1, 1, 'L'],
 ['B', 1, 0, 'M'],
 ['B', 0, 1, 'N'],
 ['B', 1, 0, 'O'],
 ['B', 0, 1, 'P'],
 ['B', 1, 1, 'Q'],
 ['B', 1, 0, 'R'],
 ['B', 0, 1, 'S'],
 ['B', 1, 0, 'T'],
 ['C', 0, 0, 'A'],
 ['C', 1, 1, 'B'],
 ['C', 0, 0, 'D'],
 ['C', 1, 1, 'E'],
 ['C', 1, 1, 'F'],
 ['C', 1, 0, 'G'],
 ['C', 1, 1, 'H'],
 ['C', 0, 1, 'I'],
 ['C', 1, 1, 'J'],
 ['C', 1, 0, 'K'],
 ['C', 0, 0, 'L'],
 ['C', 0, 0, 'M'],
 ['C', 0, 0, 'N'],
 ['C', 1, 1, 'O'],
 ['C', 0, 1,

Perceba que fazendo

$força~de~ataque~do~time = \sum força~dos~jogadores \text{ e } força~de~defesa~do~time = \sum \log{força~dos~jogadores},$

um time com maior força de ataque terá maior força de defesa, o que não é compatível com a realidade.

# Usando dois parâmetros por jogador

Cada jogador tem dois parâmetros: ataque e defesa

In [7]:
def generate_players(clubs = 20, players_per_club = 23):
    '''
    creates players
    '''
    n_players = clubs * players_per_club
    players_atk = truncnorm.rvs(0, 10, loc = 2, scale = 0.5, size = n_players)
    players_def = truncnorm.rvs(0, 10, loc = 1, scale = 0.5, size = n_players)
    players = {}
    for i in range(n_players):
        # normalizing
        players_atk[i] = players_atk[i]/players_atk[0]
        players_def[i] = players_def[i]/players_def[0]
        
        players['P{:04d}'.format(i)] = [round(players_atk[i], 8), round(players_def[i], 8)]
#         players['P{:04d}'.format(i)] = [round(players_atk[i], 8) if players_atk[i] < 100 else 100,
#                                         round(players_def[i], 8) if players_def[i] < 100 else 100]
        
    return players

def change_clubs(clubs, changes):
    '''
    change clubs
    '''
    clubs_f = {}
    changing = []
    change = 0
    for i in range(n_clubs):
        club_players = clubs[chr(i + 65)].copy()
        np.random.shuffle(club_players)
        for j in range(changes[i]):
            changing.append(club_players[j])

    np.random.shuffle(changing)
    for i in range(n_clubs):
        club_players = clubs[chr(i + 65)].copy()
        ind = []
        for player in club_players:
            if player in changing:
                ind.append(club_players.index(player))

        for j in range(len(ind)):
            club_players[ind[j]] = changing[change]
            change += 1

        clubs_f[chr(i + 65)] = club_players

    return clubs_f

def generate_clubs(players, years = 15, n_clubs = 20, players_per_club = 23):
    '''
    receives players and allocates them to clubs
    '''
    clubs = {}
    all_players = list(players)
    for year in range(years):
        line_up = {chr(65 + club) : {} for club in range(n_clubs)}
        clubs[year] = line_up
        for club in range(n_clubs):
            if year == 0:
                clubs[year][chr(65 + club)] = all_players[players_per_club * club:players_per_club * (club + 1)]

            else:
                changes = poisson.rvs(players_per_club/4, size = n_clubs)
                for i in range(n_clubs):
                    if changes[i] > players_per_club:
                        changes[i] = players_per_club

                    clubs[year] = change_clubs(clubs[year - 1], changes)
    return clubs

In [8]:
years = 15
n_clubs = 20
players_per_club = 11
players = generate_players()
clubs = generate_clubs(players,
                       years = years,
                       n_clubs = n_clubs,
                       players_per_club = players_per_club)

In [9]:
def find_forces(line_up, players):
    atk_force = 0
    def_force = 0
    for player in line_up:
        atk_force += 1.5*players[player][0]
        def_force += players[player][1]
    
    return atk_force/11, def_force/11

def model(club1, club2, players, sims = 10000):
    '''
    receives 2 clubs (line_up) and returns the most likely result, probability of each result and probability for the game
    '''
    atk1, def1 = find_forces(club1, players)
    atk2, def2 = find_forces(club2, players)
    results = np.zeros((6, 6), dtype = int)
    probs = [0, 0, 0]
    for i in range(sims):
        goals1 = poisson.rvs(atk1 / def2)
        goals2 = poisson.rvs(atk2 / def1)
        if goals1 > 5:
            goals1 = 5
        if goals2 > 5:
            goals2 = 5
        
        results[goals1, goals2] += 1
        
        if goals1 > goals2:
            probs[0] += 1
        elif goals1 == goals2:
            probs[1] += 1
        else:
            probs[2] += 1
    
    for i in range(3):
        probs[i] /= sims
        
    result = [0, 0] # Home - away
    for i in range(len(results)):
        for j in range(len(results[i])):
            if results[i, j] > results[result[0], result[1]]:
                result = [i, j]
            elif results[i, j] == results[result[0], result[1]]:
                # rare cases
                if np.random.random() > 0.5:
                    result = [i, j]
    
    return results, result, probs

In [10]:
def championship(clubs, players, sims = 10000, year = 0):
    games = generate_games()
    results = []
    line_up = []
    table = np.zeros((20, 8), dtype = int)
    # lines = clubs
    # columns = {points, games, wins, draws, defeats, goals for, goals against, goal difference}
    
    for i in range(len(games)):
        home = i
        for away in games[i]:
            away = ord(away) - 65
            
            club1 = clubs[year][chr(home + 65)].copy()
            np.random.shuffle(club1)
            home_line_up = club1[:11]
            
            club2 = clubs[year][chr(away + 65)].copy()
            np.random.shuffle(club2)
            away_line_up = club2[:11]
            
            _, result, _ = model(home_line_up, away_line_up, players, sims = sims)
            results.append([chr(home + 65), result[0], result[1], chr(away + 65)])
            line_up.append([home_line_up, away_line_up])
            
            # games
            table[home, 1] += 1
            table[away, 1] += 1
            
            # goals
            table[home, 5] += result[0]
            table[away, 5] += result[1]
            table[home, 6] += result[1]
            table[away, 6] += result[0]
            table[home, 7] += result[0] - result[1]
            table[away, 7] += result[1] - result[0]
            
            # results
            if result[0] > result[1]:
                table[home, 2] += 1
                table[home, 0] += 3
                table[away, 4] += 1
            elif result[0] == result[1]:
                table[home, 3] += 1
                table[away, 3] += 1
                table[home, 0] += 1
                table[away, 0] += 1
            else:
                table[home, 4] += 1
                table[away, 2] += 1
                table[away, 0] += 3
    
    columns_names = ['points', 'games', 'wins', 'draws', 'defeats', 'goals for', 'goals against', 'goal difference']
    
    table = pd.DataFrame(table,
                        index = [chr(i + 65) for i in range(20)],
                        columns = columns_names)
    table.sort_values(['points',  'wins', 'goal difference', 'goals for'],
                      axis = 0,
                      ascending = False,
                      inplace = True)
    
    return table, results, line_up

In [11]:
table, results, line_up = championship(clubs, players, sims = 1000, year = 0)

In [12]:
table

,points,games,wins,draws,defeats,goals for,goals against,goal difference
K,76,38,20,16,2,88,68,20
C,65,38,15,20,3,72,60,12
F,59,38,12,23,3,79,69,10
M,54,38,10,24,4,81,75,6
L,52,38,9,25,4,79,73,6
T,49,38,8,25,5,75,72,3
D,47,38,8,23,7,86,85,1
R,47,38,8,23,7,72,71,1
O,45,38,6,27,5,78,77,1
B,45,38,5,30,3,79,77,2


In [13]:
results

[['A', 2, 2, 'B'],
 ['A', 1, 1, 'C'],
 ['A', 1, 2, 'D'],
 ['A', 2, 3, 'E'],
 ['A', 2, 2, 'F'],
 ['A', 2, 2, 'G'],
 ['A', 2, 2, 'H'],
 ['A', 1, 1, 'I'],
 ['A', 2, 2, 'J'],
 ['A', 1, 2, 'K'],
 ['A', 1, 2, 'L'],
 ['A', 1, 2, 'M'],
 ['A', 1, 2, 'N'],
 ['A', 2, 2, 'O'],
 ['A', 2, 2, 'P'],
 ['A', 2, 1, 'Q'],
 ['A', 2, 2, 'R'],
 ['A', 2, 2, 'S'],
 ['A', 1, 2, 'T'],
 ['B', 2, 2, 'A'],
 ['B', 2, 2, 'C'],
 ['B', 3, 3, 'D'],
 ['B', 2, 2, 'E'],
 ['B', 2, 2, 'F'],
 ['B', 3, 2, 'G'],
 ['B', 2, 2, 'H'],
 ['B', 2, 2, 'I'],
 ['B', 2, 2, 'J'],
 ['B', 2, 2, 'K'],
 ['B', 2, 2, 'L'],
 ['B', 2, 2, 'M'],
 ['B', 2, 2, 'N'],
 ['B', 2, 2, 'O'],
 ['B', 2, 1, 'P'],
 ['B', 2, 2, 'Q'],
 ['B', 2, 2, 'R'],
 ['B', 3, 3, 'S'],
 ['B', 2, 2, 'T'],
 ['C', 2, 1, 'A'],
 ['C', 2, 2, 'B'],
 ['C', 2, 2, 'D'],
 ['C', 2, 1, 'E'],
 ['C', 1, 2, 'F'],
 ['C', 2, 1, 'G'],
 ['C', 1, 1, 'H'],
 ['C', 2, 1, 'I'],
 ['C', 2, 2, 'J'],
 ['C', 1, 1, 'K'],
 ['C', 2, 2, 'L'],
 ['C', 2, 2, 'M'],
 ['C', 2, 1, 'N'],
 ['C', 2, 1, 'O'],
 ['C', 2, 2,

In [14]:
line_up

[[['P0002',
   'P0005',
   'P0010',
   'P0000',
   'P0001',
   'P0003',
   'P0008',
   'P0006',
   'P0004',
   'P0009',
   'P0007'],
  ['P0019',
   'P0020',
   'P0021',
   'P0011',
   'P0014',
   'P0012',
   'P0018',
   'P0015',
   'P0013',
   'P0016',
   'P0017']],
 [['P0002',
   'P0005',
   'P0003',
   'P0001',
   'P0008',
   'P0006',
   'P0010',
   'P0009',
   'P0007',
   'P0000',
   'P0004'],
  ['P0028',
   'P0031',
   'P0023',
   'P0025',
   'P0024',
   'P0030',
   'P0022',
   'P0026',
   'P0032',
   'P0027',
   'P0029']],
 [['P0006',
   'P0001',
   'P0002',
   'P0000',
   'P0009',
   'P0007',
   'P0005',
   'P0004',
   'P0008',
   'P0010',
   'P0003'],
  ['P0040',
   'P0041',
   'P0038',
   'P0034',
   'P0043',
   'P0039',
   'P0036',
   'P0037',
   'P0033',
   'P0035',
   'P0042']],
 [['P0001',
   'P0003',
   'P0008',
   'P0002',
   'P0009',
   'P0005',
   'P0007',
   'P0004',
   'P0006',
   'P0010',
   'P0000'],
  ['P0048',
   'P0044',
   'P0049',
   'P0051',
   'P0045',
   'P0

# Generate Data

In [15]:
results = []
line_up = []
for year in range(years):
    table, result, squad = championship(clubs, players, sims = 1000, year = year)
    for i in range(len(result)):
        results.append(result[i])
        line_up.append(squad[i])

In [16]:
results

[['A', 2, 1, 'B'],
 ['A', 1, 2, 'C'],
 ['A', 3, 2, 'D'],
 ['A', 2, 2, 'E'],
 ['A', 2, 2, 'F'],
 ['A', 3, 2, 'G'],
 ['A', 2, 2, 'H'],
 ['A', 2, 2, 'I'],
 ['A', 2, 1, 'J'],
 ['A', 1, 2, 'K'],
 ['A', 1, 1, 'L'],
 ['A', 2, 2, 'M'],
 ['A', 2, 2, 'N'],
 ['A', 2, 2, 'O'],
 ['A', 2, 2, 'P'],
 ['A', 2, 2, 'Q'],
 ['A', 1, 3, 'R'],
 ['A', 1, 2, 'S'],
 ['A', 2, 2, 'T'],
 ['B', 1, 1, 'A'],
 ['B', 2, 2, 'C'],
 ['B', 2, 2, 'D'],
 ['B', 2, 3, 'E'],
 ['B', 2, 2, 'F'],
 ['B', 2, 2, 'G'],
 ['B', 2, 2, 'H'],
 ['B', 2, 3, 'I'],
 ['B', 3, 2, 'J'],
 ['B', 1, 2, 'K'],
 ['B', 2, 2, 'L'],
 ['B', 2, 2, 'M'],
 ['B', 3, 2, 'N'],
 ['B', 3, 2, 'O'],
 ['B', 2, 2, 'P'],
 ['B', 2, 1, 'Q'],
 ['B', 2, 2, 'R'],
 ['B', 2, 2, 'S'],
 ['B', 2, 2, 'T'],
 ['C', 2, 2, 'A'],
 ['C', 2, 1, 'B'],
 ['C', 2, 2, 'D'],
 ['C', 2, 3, 'E'],
 ['C', 1, 2, 'F'],
 ['C', 3, 1, 'G'],
 ['C', 2, 1, 'H'],
 ['C', 2, 2, 'I'],
 ['C', 2, 2, 'J'],
 ['C', 2, 2, 'K'],
 ['C', 2, 1, 'L'],
 ['C', 2, 2, 'M'],
 ['C', 2, 1, 'N'],
 ['C', 2, 1, 'O'],
 ['C', 3, 2,

In [17]:
line_up

[[['P0002',
   'P0003',
   'P0006',
   'P0004',
   'P0000',
   'P0009',
   'P0005',
   'P0008',
   'P0007',
   'P0010',
   'P0001'],
  ['P0011',
   'P0020',
   'P0017',
   'P0013',
   'P0014',
   'P0018',
   'P0016',
   'P0021',
   'P0015',
   'P0019',
   'P0012']],
 [['P0008',
   'P0003',
   'P0004',
   'P0006',
   'P0002',
   'P0000',
   'P0005',
   'P0001',
   'P0009',
   'P0010',
   'P0007'],
  ['P0022',
   'P0029',
   'P0025',
   'P0028',
   'P0031',
   'P0023',
   'P0032',
   'P0027',
   'P0026',
   'P0030',
   'P0024']],
 [['P0009',
   'P0006',
   'P0010',
   'P0000',
   'P0003',
   'P0008',
   'P0007',
   'P0004',
   'P0001',
   'P0005',
   'P0002'],
  ['P0035',
   'P0042',
   'P0033',
   'P0040',
   'P0036',
   'P0041',
   'P0034',
   'P0038',
   'P0039',
   'P0037',
   'P0043']],
 [['P0000',
   'P0006',
   'P0009',
   'P0007',
   'P0003',
   'P0008',
   'P0001',
   'P0004',
   'P0002',
   'P0010',
   'P0005'],
  ['P0044',
   'P0045',
   'P0047',
   'P0046',
   'P0053',
   'P0

# Engenharia reversa

In [18]:
def find_forces(line_up, players):
    atk_force = 0
    def_force = 0
    for player in line_up:
        atk_force += 1.5*players[player][0]
        def_force += players[player][1]
    
    return atk_force/11, def_force/11

def find_players(players_er, codes):
    players = {}
    for code in codes:
        if code not in players:
            players[code] = [players_er[2 * codes.index(code)], players_er[2 * codes.index(code) + 1]]
    
    return players

def likelihood(players_er, codes, results, line_up):
    '''
    players : parameter list
    results : list
    line_up : list
    '''
    players_er = find_players(players_er, codes)
    
    # normalizing
    for player in players_er:
        players_er[player][0] = players_er[player][0]/players_er['P0000'][0]
        players_er[player][1] = players_er[player][1]/players_er['P0000'][1]
        
    loglikelihood = 0
    for game in line_up:               
        ind = line_up.index(game)
        atk1, def1 = find_forces(game[0], players_er)
        atk2, def2 = find_forces(game[1], players_er)
        
        loglikelihood += poisson.logpmf(results[ind][1], atk1 / def2)
        loglikelihood += poisson.logpmf(results[ind][2], atk2 / def1)
        
    global k
    if k % 1000 == 0:
        print('Iteration: {}\nValue: {}\n'.format(k, -loglikelihood))
    k += 1
    
    return -loglikelihood

def likelihood_dic(players, results, line_up):
    '''
    players : parameter list
    results : list
    line_up : list
    '''
    loglikelihood = 0
    for game in line_up:               
        ind = line_up.index(game)
        atk1, def1 = find_forces(game[0], players)
        atk2, def2 = find_forces(game[1], players)
        
        loglikelihood += poisson.logpmf(results[ind][1], atk1 / def2)
        loglikelihood += poisson.logpmf(results[ind][2], atk2 / def1)
    
    return -loglikelihood

In [19]:
k = 0
players_er = {}
codes = []
for game in line_up:
    for player in game[0]:
        if player not in players_er:
            players_er[player] = [truncnorm.rvs(0, 10, loc = 3, scale = 1), truncnorm.rvs(0, 10, loc = 1, scale = 1)]
            codes.append(player)
    for player in game[1]:
        if player not in players_er:
            players_er[player] = [truncnorm.rvs(0, 10, loc = 1, scale = 1), truncnorm.rvs(0, 10, loc = 1, scale = 1)]
            codes.append(player)

# players_er = [players_er[player] for player in players_er]
aux = players_er.copy()
players_er = []
for i in aux:
    players_er.append(aux[i][0])
    players_er.append(aux[i][1])

lik = likelihood(players_er, codes, results, line_up)
lik

Iteration: 0
Value: 16711.276900340716



16711.276900340716

In [20]:
result = minimize(likelihood, players_er, args = (codes, results, line_up), bounds = [(0, None) for player in players_er])

Iteration: 1000
Value: 15654.570135665834

Iteration: 2000
Value: 15547.556188213175

Iteration: 3000
Value: 15486.585473977331

Iteration: 4000
Value: 15471.668437607377

Iteration: 5000
Value: 15458.189573916296

Iteration: 6000
Value: 15449.445941956443

Iteration: 7000
Value: 15447.731067814113

Iteration: 8000
Value: 15443.661978879592

Iteration: 9000
Value: 15442.485718799991

Iteration: 10000
Value: 15441.559566594702

Iteration: 11000
Value: 15440.31143971504

Iteration: 12000
Value: 15438.768625982448

Iteration: 13000
Value: 15438.31937590238

Iteration: 14000
Value: 15437.970629191028

Iteration: 15000
Value: 15437.505005710776



In [21]:
result

      fun: 15437.505005710747
 hess_inv: <440x440 LbfgsInvHessProduct with dtype=float64>
      jac: array([-3.63797883e-04,  3.45789888e-01, -4.11091608e-02,  5.23868951e-01,
        2.91038306e-03,  3.05954020e-01, -1.78260963e-02,  3.59796106e-01,
        1.88993000e-01, -3.66762836e+00, -5.29325920e-02,  2.50656741e-01,
       -7.63975554e-03,  9.98625189e-02, -2.00088836e-03,  2.57568901e-01,
       -6.29370337e-02,  2.35922927e-01, -7.27595766e-02,  2.05545804e-02,
       -2.83762349e-02,  1.65528037e-02, -2.98314264e-02, -7.83984438e-02,
       -9.56788432e-02,  2.81943359e-02, -6.25732359e-02,  1.74804882e-01,
       -2.23735698e-02, -1.52795111e-02, -1.83717931e-02,  1.81898941e-04,
       -4.25643523e-02,  8.91304813e-03, -3.18323148e-02, -3.76530809e-02,
       -5.03860068e-02, -5.09317036e-02, -4.98403100e-02, -1.28966349e-01,
       -3.03771232e-02, -1.43518265e-01, -6.27551348e-02,  6.91215978e-02,
       -2.63753465e-02,  8.45830078e-02, -1.11140253e-01,  1.35696610e-01,

In [22]:
result.nit

33

In [23]:
result.x

array([3.04874357, 1.96039193, 4.99170615, 3.36176447, 4.8813939 ,
       1.93103884, 5.6000994 , 2.2925701 , 1.70665217, 1.05430593,
       1.8482657 , 2.37932593, 2.69412247, 2.4144555 , 2.33019171,
       2.72354697, 1.81238722, 2.62993112, 3.40596405, 1.96505423,
       1.27409753, 2.20462536, 1.58123946, 2.82507725, 2.63344245,
       1.8390636 , 1.91025242, 0.        , 2.8787542 , 1.99353878,
       3.55880954, 1.40331087, 1.9652078 , 1.36991503, 3.40453993,
       1.21433397, 2.18780369, 2.3747756 , 2.39594473, 2.67771306,
       2.74576366, 3.79975899, 2.34167875, 1.16709533, 3.60694867,
       3.33533177, 3.28858473, 2.00394395, 1.80279945, 2.49429798,
       2.45308487, 2.36798852, 2.83711259, 1.36202682, 2.43939218,
       2.88084174, 1.47658131, 3.29009765, 2.65659001, 2.36826661,
       3.04459602, 2.14150823, 2.26018439, 1.1474556 , 2.82697544,
       1.53213659, 1.50363074, 0.98311286, 2.58505119, 2.49054501,
       2.32123517, 1.61171702, 3.54794651, 1.57150755, 1.95429

In [24]:
players_er = find_players(result.x, codes)
players_er = collections.OrderedDict(sorted(players_er.items()))
players_er

OrderedDict([('P0000', [1.7066521657547422, 1.0543059349028847]),
             ('P0001', [1.2740975272095485, 2.2046253589726352]),
             ('P0002', [3.0487435698087557, 1.9603919258691627]),
             ('P0003', [4.991706149615297, 3.3617644748224706]),
             ('P0004', [5.600099401594242, 2.292570103964672]),
             ('P0005', [2.6941224652520224, 2.4144554974939414]),
             ('P0006', [4.881393901025443, 1.9310388379931296]),
             ('P0007', [1.8123872169352748, 2.6299311235240963]),
             ('P0008', [2.330191710396009, 2.723546971099434]),
             ('P0009', [1.848265701506246, 2.3793259288909967]),
             ('P0010', [3.405964048487344, 1.9650542285724253]),
             ('P0011', [1.5812394590823007, 2.825077254986533]),
             ('P0012', [2.3416787457905164, 1.167095325724282]),
             ('P0013', [2.8787541952425606, 1.9935387750669546]),
             ('P0014', [3.55880954153792, 1.4033108696105894]),
             ('P0015',

In [25]:
players

{'P0000': [1.0, 1.0],
 'P0001': [2.04566354, 1.41291168],
 'P0002': [2.06066851, 1.58413645],
 'P0003': [2.445591, 2.42094835],
 'P0004': [2.12772039, 1.12848182],
 'P0005': [2.18524814, 1.6281186],
 'P0006': [2.01382751, 1.02038602],
 'P0007': [2.00055893, 1.55359818],
 'P0008': [2.15965288, 1.93200154],
 'P0009': [2.01623542, 1.69062407],
 'P0010': [2.23467286, 1.52547039],
 'P0011': [2.4477419, 2.52946402],
 'P0012': [2.2659025, 1.50566063],
 'P0013': [2.1517952, 1.24168121],
 'P0014': [2.55428703, 1.14333682],
 'P0015': [2.5364521, 1.50099219],
 'P0016': [2.17957052, 1.1563145],
 'P0017': [2.71514468, 1.09295117],
 'P0018': [2.0819439, 1.04588883],
 'P0019': [2.05594825, 1.46860971],
 'P0020': [2.36997801, 1.14087656],
 'P0021': [2.26320557, 1.42555111],
 'P0022': [2.5287056, 1.81591185],
 'P0023': [2.16709064, 1.45047346],
 'P0024': [2.01640981, 1.04954727],
 'P0025': [2.42857532, 1.71593837],
 'P0026': [2.48337229, 2.1831866],
 'P0027': [2.34917482, 1.85421045],
 'P0028': [2.4767

In [26]:
lik = likelihood_dic(players, results, line_up)
lik

15965.41716990267

In [27]:
result.fun

15437.505005710747

In [28]:
table, results, line_up = championship(clubs, find_players(result.x, codes), sims = 1000, year = 0)

In [29]:
table

,points,games,wins,draws,defeats,goals for,goals against,goal difference
A,91,38,27,10,1,70,44,26
K,79,38,22,13,3,66,46,20
C,71,38,18,17,3,64,49,15
L,63,38,15,18,5,57,47,10
R,62,38,14,20,4,59,49,10
F,55,38,12,19,7,56,50,6
Q,51,38,11,18,9,60,58,2
N,48,38,11,15,12,56,57,-1
M,47,38,11,14,13,59,61,-2
T,47,38,9,20,9,54,54,0
